**TODO:**
- **Critical:** dive into inflation expectations in the first half of 20th century (golden standard?)
- Test on data of different countries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from scipy.optimize import curve_fit
from typing import Sequence
import patsy

np.set_printoptions(suppress=True)

#%matplotlib widget

In [3]:
dataset = pd.read_csv("dataset.csv",index_col="year")
dataset.head()

,inflation_rate,mean_unemployment,expected_inflation
year,,,
1948-12-31,0.016878,0.037500,0.088372
1949-12-31,-0.016667,0.060500,0.016878
1950-12-31,0.063830,0.052083,-0.016667
1951-12-31,0.043307,0.032833,0.063830
1952-12-31,0.007547,0.030250,0.043307


**Developing a model to describe the phenomenon**

In [12]:
y, X = patsy.dmatrices("inflation_rate ~ mean_unemployment + expected_inflation + 0", dataset)

Linear regeression

In [25]:
model = LinearRegression()
results = model.fit(X,y)
coefs = results.coef_.squeeze()
intercept = results.intercept_

print(f"Coefficients:")
print(pd.Series(coefs,index=X.design_info.column_names))
print(f"\nR-square: {round(model.score(X,y),4)}")

Coefficients:
mean_unemployment    -0.079073
expected_inflation    0.742149
dtype: float64

R-square: 0.405


Non linear curve (a*y)/(b*x) + c

In [ ]:
# TODO
def curve_func(X, a, b):
    x, y = X
    return a*(y/x) + b

popt, pcov = curve_fit(curve_func, X, y, p0=[1,0])

print("Coefficients: " + str(popt))
print("R^2 = " + str(r_squared(curve_func, X, y, *popt)))

In [ ]:
# plotting the data points and the fitted curve
fig = plt.figure(clear=True)
ax = fig.add_subplot(projection='3d')
fontdict = {
    "fontweight":"bold",
    "size":5,
    "fontvariant":"normal"
}

ax.view_init(25, -140, 0)
ax.scatter(xs=dataset["mean_unemployment"], ys=dataset["expected_inflation"], zs=dataset["inflation_rate"], s=4, alpha=1)

# Figure
fig.set_label("Fitted unemployment, expected inflation and the actual inflation")

# X-axis
ax.set_xlabel("Mean Unemployment (X)",fontdict=fontdict)
ax.tick_params(axis="x",labelsize=7)

# Y-axis
ax.set_ylabel("Expected Inflation Rate (Y)",fontdict=fontdict);
ax.tick_params(axis="y",labelsize=7)

# Z-axis
ax.set_zlabel("Actual Inflation Rate (Z)",fontdict=fontdict)
ax.tick_params(axis="z",labelsize=7)

# plotting the function
x_range = np.linspace(dataset["mean_unemployment"].min(), dataset["mean_unemployment"].max(), num=100)
y_range = np.linspace(dataset["expected_inflation"].min(), dataset["expected_inflation"].max(), num=100) 
xx, yy = np.meshgrid(x_range, y_range)
zz = curve_func((xx, yy), *popt)
ax.plot_surface(xx, yy, zz, color='green', alpha=0.5);